In [1]:
import pymysql
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import getpass
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix
password = getpass.getpass()

········


In [2]:
# get the data
connection_string = 'mysql+pymysql://root:' + password + '@localhost/sakila'
engine = create_engine(connection_string)

In [3]:
'''query1 = SELECT f.film_id, f.title, f.release_year, f.length, f.rating, c.name AS category_name,
        r.rental_date,
        COUNT(*) AS num_rentals FROM film f 
    INNER JOIN film_category fc ON f.film_id = fc.film_id
    INNER JOIN category c ON fc.category_id = c.category_id
    INNER JOIN inventory i ON f.film_id = i.film_id
    INNER JOIN rental r ON i.inventory_id = r.inventory_id WHERE MONTH(r.rental_date) = 5
GROUP BY f.film_id, f.title, f.release_year, f.length, f.rating, c.name, r.rental_date;'''

'query1 = SELECT f.film_id, f.title, f.release_year, f.length, f.rating, c.name AS category_name,\n        r.rental_date,\n        COUNT(*) AS num_rentals FROM film f \n    INNER JOIN film_category fc ON f.film_id = fc.film_id\n    INNER JOIN category c ON fc.category_id = c.category_id\n    INNER JOIN inventory i ON f.film_id = i.film_id\n    INNER JOIN rental r ON i.inventory_id = r.inventory_id WHERE MONTH(r.rental_date) = 5\nGROUP BY f.film_id, f.title, f.release_year, f.length, f.rating, c.name, r.rental_date;'

In [4]:
query2= '''SELECT f.title,
       MIN(r.rental_date) AS rental_date,
       c.name AS category_name,
       CASE WHEN YEAR(MIN(r.rental_date)) = 2005 AND MONTH(MIN(r.rental_date)) = 5
            THEN True
            ELSE False
       END AS rented_in_may,
       f.length,
       f.rental_rate,
       f.rating
FROM film f
JOIN inventory i ON f.film_id = i.film_id
JOIN rental r ON i.inventory_id = r.inventory_id
JOIN film_category fc ON f.film_id = fc.film_id
JOIN category c ON fc.category_id = c.category_id
GROUP BY f.title, c.name, f.length, f.rental_rate, f.rating;'''

In [5]:
#q1 = pd.read_sql_query(query1, engine)
#q1.head()

In [6]:
#q1.num_rentals.value_counts()

In [7]:
#q1.shape

In [8]:
'''query2 = SELECT DISTINCT f.title, COUNT(r.rental_id) > 0 AS rented_in_may 
FROM film f 
LEFT JOIN inventory i ON f.film_id = i.film_id
LEFT JOIN rental r ON i.inventory_id = r.inventory_id AND r.rental_date BETWEEN '2005-05-01' AND '2005-05-31'
GROUP BY f.title;
'''

"query2 = SELECT DISTINCT f.title, COUNT(r.rental_id) > 0 AS rented_in_may \nFROM film f \nLEFT JOIN inventory i ON f.film_id = i.film_id\nLEFT JOIN rental r ON i.inventory_id = r.inventory_id AND r.rental_date BETWEEN '2005-05-01' AND '2005-05-31'\nGROUP BY f.title;\n"

In [9]:
q2 = pd.read_sql(query2, engine)
q2.head()

,title,rental_date,category_name,rented_in_may,length,rental_rate,rating
0,AMADEUS HOLY,2005-05-28 12:08:37,Action,1,113,0.99,PG
1,AMERICAN CIRCUS,2005-05-25 21:58:58,Action,1,129,4.99,R
2,ANTITRUST TOMATOES,2005-05-31 16:48:43,Action,1,168,2.99,NC-17
3,BAREFOOT MANCHURIAN,2005-05-26 20:57:00,Action,1,129,2.99,G
4,BERETS AGENT,2005-05-29 23:37:00,Action,1,77,2.99,PG-13


In [10]:
q2['rented_in_may'].unique()

array([1, 0], dtype=int64)

In [11]:
q2.shape

(958, 7)

In [12]:
#

In [13]:
#data = pd.merge(q1, q2, on='title', how='right')

In [14]:
data = q2

In [15]:
#

In [16]:
data.isnull().sum()

title            0
rental_date      0
category_name    0
rented_in_may    0
length           0
rental_rate      0
rating           0
dtype: int64

In [17]:
#data.dropna(thresh=len(data.columns) - 1, inplace=True)
#data.dropna(inplace=True)

In [18]:
data.shape

(958, 7)

In [19]:
data

,title,rental_date,category_name,rented_in_may,length,rental_rate,rating
0,AMADEUS HOLY,2005-05-28 12:08:37,Action,1,113,0.99,PG
1,AMERICAN CIRCUS,2005-05-25 21:58:58,Action,1,129,4.99,R
2,ANTITRUST TOMATOES,2005-05-31 16:48:43,Action,1,168,2.99,NC-17
3,BAREFOOT MANCHURIAN,2005-05-26 20:57:00,Action,1,129,2.99,G
4,BERETS AGENT,2005-05-29 23:37:00,Action,1,77,2.99,PG-13
...,...,...,...,...,...,...,...
953,VALENTINE VANISHING,2005-05-25 19:30:46,Travel,1,48,0.99,PG-13
954,WINDOW SIDE,2005-06-15 03:50:36,Travel,0,85,2.99,R
955,WOLVES DESIRE,2005-06-15 06:54:53,Travel,0,55,0.99,NC-17
956,WORKER TARZAN,2005-06-20 17:26:05,Travel,0,139,2.99,R


In [20]:
data.isnull().sum()

title            0
rental_date      0
category_name    0
rented_in_may    0
length           0
rental_rate      0
rating           0
dtype: int64

In [21]:
#data.drop(['rental_date'], axis=1, inplace=True)

In [22]:
#corr_matrix=data.corr(method='pearson')  # default
#fig, ax = plt.subplots(figsize=(10, 8))
#ax = sns.heatmap(corr_matrix, annot=True)
#plt.show()

In [23]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 958 entries, 0 to 957
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   title          958 non-null    object        
 1   rental_date    958 non-null    datetime64[ns]
 2   category_name  958 non-null    object        
 3   rented_in_may  958 non-null    int64         
 4   length         958 non-null    int64         
 5   rental_rate    958 non-null    float64       
 6   rating         958 non-null    object        
dtypes: datetime64[ns](1), float64(1), int64(2), object(3)
memory usage: 52.5+ KB


In [24]:
y = data['rented_in_may']
X = data.drop(['rented_in_may'], axis=1)

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1337)

In [25]:
X_train_categorical = X_train.select_dtypes('object')
X_test_categorical  = X_test.select_dtypes('object')

from sklearn.preprocessing import OneHotEncoder


X_train_cat = pd.get_dummies(X_train_categorical, 
                             columns=X_train_categorical.columns,
                             drop_first=True).reset_index()
X_test_cat = pd.get_dummies(X_test_categorical, 
                            columns=X_test_categorical.columns,
                            drop_first=True).reset_index()
X_train_cat.head()

,index,title_ADAPTATION HOLES,title_AFRICAN EGG,title_AGENT TRUMAN,title_AIRPLANE SIERRA,title_ALABAMA DEVIL,title_ALLEY EVOLUTION,title_ALONE TRIP,title_ALTER VICTORY,title_AMADEUS HOLY,...,category_name_Horror,category_name_Music,category_name_New,category_name_Sci-Fi,category_name_Sports,category_name_Travel,rating_NC-17,rating_PG,rating_PG-13,rating_R
0,70,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,102,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,515,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,120,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,133,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [26]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler
# from sklearn.preprocessing import StandardScaler

X_train_num = X_train.select_dtypes(include = np.number)
X_test_num  = X_test.select_dtypes(include = np.number)

# Scaling data
transformer = MinMaxScaler().fit(X_train_num) # need to keep transformer

X_train_normalized = transformer.transform(X_train_num)
X_test_normalized  = transformer.transform(X_test_num)

X_train_norm = pd.DataFrame(X_train_normalized, columns=X_train_num.columns)
X_test_norm  = pd.DataFrame(X_test_normalized, columns=X_test_num.columns)

In [27]:
X_train_norm.describe()

,length,rental_rate
count,766.000000,766.000000
mean,0.506086,0.490862
std,0.289162,0.410541
min,0.000000,0.000000
25%,0.266187,0.000000
50%,0.503597,0.500000
75%,0.746403,1.000000
max,1.000000,1.000000


In [28]:
X_train_norm.columns = X_train_num.columns
X_train_norm.head()

,length,rental_rate
0,0.136691,0.0
1,0.280576,0.5
2,0.971223,0.0
3,0.690647,0.0
4,0.697842,0.0


In [29]:
#data = pd.get_dummies(data, columns=['rating', 'category_name'])

In [30]:
#X_train_categorical = X_train.select_dtypes('object')
#X_test_categorical  = X_test.select_dtypes('object')

#from sklearn.preprocessing import OneHotEncoder

#categories_values = [list(data[col].unique()) for col in X_train_categorical.columns]
#encoder = OneHotEncoder(drop='first', handle_unknown='ignore')
#encoder.fit(X_train_categorical)

#X_train_cat_np = encoder.transform(X_train_categorical).toarray()
#X_test_cat_np  = encoder.transform(X_test_categorical).toarray()

#X_train_cat = pd.DataFrame(X_train_cat_np, columns=encoder.get_feature_names_out())
#X_test_cat  = pd.DataFrame(X_test_cat_np,  columns=encoder.get_feature_names_out())

#X_train_cat = pd.get_dummies(X_train_categorical, 
#                             columns=['type', 'operation', 'k_symbol', 'duration'],
#                             drop_first=True)
#X_test_cat = pd.get_dummies(X_test_categorical, 
#                             columns=['type', 'operation', 'k_symbol', 'duration'],
#                             drop_first=True)

#X_train_cat.head()

In [31]:
#X_train_categorical = X_train.select_dtypes('object')
#X_test_categorical  = X_test.select_dtypes('object')

#from sklearn.preprocessing import OneHotEncoder


#X_train_cat = pd.get_dummies(X_train_categorical, 
   #                          columns=X_train_categorical.columns,
  #                           drop_first=True)
#X_test_cat = pd.get_dummies(X_test_categorical, 
 #                           columns=X_test_categorical.columns,
#                            drop_first=True)
#X_train_cat.head()

In [32]:
X_train_cat.isna().sum()

index                     0
title_ADAPTATION HOLES    0
title_AFRICAN EGG         0
title_AGENT TRUMAN        0
title_AIRPLANE SIERRA     0
                         ..
category_name_Travel      0
rating_NC-17              0
rating_PG                 0
rating_PG-13              0
rating_R                  0
Length: 785, dtype: int64

In [33]:
X_train_cat

,index,title_ADAPTATION HOLES,title_AFRICAN EGG,title_AGENT TRUMAN,title_AIRPLANE SIERRA,title_ALABAMA DEVIL,title_ALLEY EVOLUTION,title_ALONE TRIP,title_ALTER VICTORY,title_AMADEUS HOLY,...,category_name_Horror,category_name_Music,category_name_New,category_name_Sci-Fi,category_name_Sports,category_name_Travel,rating_NC-17,rating_PG,rating_PG-13,rating_R
0,70,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,102,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,515,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,120,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,133,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
761,167,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
762,232,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
763,860,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
764,189,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [34]:
X_train_transformed = pd.concat([X_train_norm, X_train_cat], axis=1)


In [35]:
X_train_transformed

,length,rental_rate,index,title_ADAPTATION HOLES,title_AFRICAN EGG,title_AGENT TRUMAN,title_AIRPLANE SIERRA,title_ALABAMA DEVIL,title_ALLEY EVOLUTION,title_ALONE TRIP,...,category_name_Horror,category_name_Music,category_name_New,category_name_Sci-Fi,category_name_Sports,category_name_Travel,rating_NC-17,rating_PG,rating_PG-13,rating_R
0,0.136691,0.0,70,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,0.280576,0.5,102,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0.971223,0.0,515,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0.690647,0.0,120,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0.697842,0.0,133,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
761,0.776978,0.5,167,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
762,0.690647,0.5,232,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
763,0.503597,0.5,860,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
764,0.992806,0.5,189,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [36]:
display(y_train)

70     1
102    1
515    1
120    1
133    0
      ..
167    1
232    1
860    1
189    0
151    1
Name: rented_in_may, Length: 766, dtype: int64

In [37]:
X_train_transformed

,length,rental_rate,index,title_ADAPTATION HOLES,title_AFRICAN EGG,title_AGENT TRUMAN,title_AIRPLANE SIERRA,title_ALABAMA DEVIL,title_ALLEY EVOLUTION,title_ALONE TRIP,...,category_name_Horror,category_name_Music,category_name_New,category_name_Sci-Fi,category_name_Sports,category_name_Travel,rating_NC-17,rating_PG,rating_PG-13,rating_R
0,0.136691,0.0,70,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,0.280576,0.5,102,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0.971223,0.0,515,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0.690647,0.0,120,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0.697842,0.0,133,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
761,0.776978,0.5,167,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
762,0.690647,0.5,232,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
763,0.503597,0.5,860,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
764,0.992806,0.5,189,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [38]:
X_train_transformed.dtypes

length                    float64
rental_rate               float64
index                       int64
title_ADAPTATION HOLES      uint8
title_AFRICAN EGG           uint8
                           ...   
category_name_Travel        uint8
rating_NC-17                uint8
rating_PG                   uint8
rating_PG-13                uint8
rating_R                    uint8
Length: 787, dtype: object

In [39]:
y_train

70     1
102    1
515    1
120    1
133    0
      ..
167    1
232    1
860    1
189    0
151    1
Name: rented_in_may, Length: 766, dtype: int64

In [40]:
from sklearn.linear_model import LogisticRegression

classification = LogisticRegression(random_state=0, solver='saga',
                  multi_class='multinomial')

classification.fit(X_train_transformed, y_train)

C:\Users\Playtime\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


LogisticRegression(multi_class='multinomial', random_state=0, solver='saga')

In [41]:
y_pred = classification.predict(X_train_transformed)
accuracy = accuracy_score(y_train, y_pred)
conf_matrix = confusion_matrix(y_train, y_pred)
print("Accuracy:", accuracy)
print("Confusion Matrix:")
print(conf_matrix)

Accuracy: 0.7310704960835509
Confusion Matrix:
[[  0 206]
 [  0 560]]
